In [1]:
import json
import os
import math
import glob
import librosa
import speech_recognition as sr
from flask import Flask, jsonify, request
from flask_cors import cross_origin
from keras.models import load_model
import numpy as np

from gensim.models.fasttext import FastText
model = FastText.load_fasttext_format('cc.en.300.bin')

c:\users\pc06\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  del sys.path[0]


In [ ]:
level1=0
level2=0
level3=0
level4=0
level5=0

# All_file = glob.glob('C:/Users/PC06/Desktop/語音進度/will smith/0/*')
file_name = []

def load_data_test(data_path):
    with open(data_path, "r",encoding="utf-8") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    X = X[..., np.newaxis]
    X = X[:,np.newaxis]
    
    return X, y


def score_(point):
    global level1, level2, level3, level4, level5
    if point >=0 and point <=20:
        level1+=1
        return ('你的美式口音還需要多加強')
    elif point >=21 and point <=40:
        level2+=1
        return ('你的美式口音有抓到一點訣竅，但還有很多細節沒有到位，多多練習會更好哦')
    elif point >=41 and point <=60:
        level3+=1
        return ('你的美式口音表現很不錯, 但一些口音細節還需要加強')
    elif point >=61 and point <=80:
        level4+=1
        return ('你的美式口音整理表現得很好、細節也有到位, 多練習就完美囉 ')
    elif point >=81 and point <=100:
        level5+=1
        return ('你的美式口英非常標準, 恭喜你')
    
def pika(point):
    if point >= 0 and point <=50:
        return "你離偶像還有一段距離哦!"
    elif point >= 50 and point <=100:
        return "哇, 跟李奧納多的聲音越來越像囉!"

def prediction(X, model_load):
    score_sum=0
    name_score=0
    point_list=[]
    for i, pre in enumerate(X):
        prediction = model_load.predict(pre)
        # print("prediction: ",prediction)
        predictied_index = np.argmax(prediction, axis=1)
        if int(predictied_index[0]) == 0:
            predictied_index = np.argmin(prediction, axis=1)
        score = (prediction[0][predictied_index])
        # # print(i, predictied_index)
#         print("prediction: ", prediction)
        score_sum += score
        if (i+1)%4==0:
            print((score_sum/4)*100)
            response = int((score_sum/4)*100)
            point_list.append(response)
            score_sum=0
            name_score+=1
    return score_(response)
#     return score(response)
    ########  回應   ##########
# PATH
def get_mfcc():
    global All_file, file_name
    data = {
    "name":[],
    "labels": [],
    "mfcc": []
    }
    JSON_PATH = "./audio.json"
    SAMPLE_RATE = 16000
    TRACK_DURATION = 10   # measured in seconds   ###### 10 、16秒兩個版本 ######
    SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION

    # PARAMS
    num_mfcc=13
    n_fft=2048
    hop_length=512
    num_segments=4
    samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
    num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)
    
#     file = All_file[n]
#     All_file[n] = All_file[n].split('\\')[-1]
#     file_name.append(All_file[n].split('.')[0])
    ####  檔案放置路徑, 需要辨識的音檔 ######
    # user files transform
#     All_file = glob.glob('C:/Users/PC06/Desktop/pikachu training/first_test/*')
    file = "./voice/translate_tts.mp3"
    signal, sample_rate = librosa.load(file, sr=SAMPLE_RATE)

    # process all segments of audio file
    for id, d in enumerate(range(num_segments)):
        # calculate start and finish sample for current segment
        start = samples_per_segment * d
        finish = start + samples_per_segment

        # extract mfcc
        mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft,
                                    hop_length=hop_length)
#         print("-----------------------------")
#         print("id", id+1)
#         print("mfcc: ", mfcc)
#         print("-----------------------------")
        
#         true_feature = 0
#         null_feature = 0
#         for n in mfcc[0]:
#             if mfcc[0] <=40:
                
#             else:
        mfcc = mfcc.T
        mfcc_cut = mfcc.copy()
        mfcc_cut[abs(mfcc_cut)<10]=0
        # store only mfcc feature with expected number of vectors
        if len(mfcc_cut) == num_mfcc_vectors_per_segment:
            # print(mfcc)
            name = file.split(".")[1].split(".mp3")[0]
            if (id+1)%4 == 0:
                data["name"].append(name)
            data["mfcc"].append(mfcc.tolist())
            data["labels"].append(0)

    with open("audio.json", "w",encoding="utf-8") as fp:
        json.dump(data, fp, indent=4)

    return file_name

## 11/30新增
def get_wave_word(file_path):
    r = sr.Recognizer()                        #預設辨識英文
    with sr.WavFile(file_path) as source:  #讀取wav檔
        audio = r.record(source)
    try:
        return "Transcription: " + r.recognize_google(audio,language="en")
                                              #使用Google的服務
    except LookupError:
        return "請再唸一次"
def check_your_voice_Volume():
    normal_sound=0
    too_low_sound=0
    ## check your talking!!
    file_path = "audio.wav"
    teacher = get_wave_word("C:/Users/PC06/Downloads/lili colins/1/Lily Collins Discusses Filming Tolkien and Ted Bundy Biopic This Morning_1.wav")
    student = get_wave_word(file_path)
    print(teacher, student)
    point = model.wv.similarity(teacher, student)
    if point <= 0.8:
        return "妳唸得不太正確, 請在唸一次!!"

In [ ]:
# # 流程 = 語音接收-->儲存-->mfcc轉換-->判別--回傳結果>
if __name__ == '__main__':
    TEST_PATH = "./audio.json"
    check_your_voice_Volume()
#     for i,  n in enumerate(range(len(All_file))):
    get_mfcc()
    # TEST_PATH = "./json/test_v4.json"
    X, y = load_data_test(TEST_PATH)
    # 16s "Model.h5" # "model_version_v7.h5"
    model_load = load_model("./model/New 0.341 - 0.9212.h5")
    response = prediction(X, model_load)
    print(file_name[i], response)